In [22]:
import pandas as pd 
from sklearn.pipeline import Pipeline, make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import (precision_score, f1_score, recall_score, accuracy_score, 
average_precision_score ,auc,roc_auc_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier , StackingClassifier
from sklearn.model_selection import train_test_split

In [66]:
data= pd.read_csv("SMOTE_dataset.csv", index_col = 0)
data.head()

,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,charging_rate_lag3,avg_time_discharging_lag1,avg_time_discharging_lag2,avg_time_discharging_lag3,discharging_rate_lag3,charge_cycle_time_below_12,avg_time_charging_lag4,...,discharging_rate_lag6_median,discharging_rate_lag8_median,avg_time_charging_lag12_random,avg_time_discharging_lag12_random,avg_time_charging_lag13_random,avg_time_discharging_lag13_random,avg_time_charging_lag14_random,avg_time_discharging_lag14_random,Days,fail_7
0,0.044546,0.409384,0.080534,0.890524,0.037009,0.828327,0.136601,0.365791,False,0.024585,...,0.622073,0.747085,0.024805,0.041633,0.024729,0.028677,0.028984,0.042778,0.686567,0
1,0.432996,0.041909,0.076458,0.891102,0.597424,0.053068,0.047526,0.384668,False,0.129345,...,0.543742,0.748188,0.093281,0.168223,0.176043,0.340139,0.030757,0.065392,0.582090,0
2,0.049358,0.040683,0.055336,0.884205,0.044314,0.045057,0.028665,0.340314,False,0.267046,...,0.605585,0.753230,0.026202,0.038698,0.022890,0.032476,0.221858,0.385565,0.562189,0
3,0.038870,0.396504,0.441206,0.890409,0.032779,0.688786,0.558263,0.382947,False,0.021194,...,0.599664,0.749081,0.021311,0.033805,0.101397,0.176052,0.028452,0.035711,0.711443,0
4,0.044299,0.431156,0.056695,0.889985,0.040085,0.643910,0.025490,0.386964,True,0.452949,...,0.605451,0.721609,0.027949,0.045725,0.210149,0.452759,0.176033,0.347875,0.800995,0


In [89]:
lst = []
for a in data:
    if "avg_time_charging_lag" in a:
        lst.append(a)
lst.sort()
lst= [lst[0]]+lst[6:]+lst[1:6]
lst.append("fail_7")
time_series = pd.DataFrame(data[lst])
time_series.head()


,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,avg_time_charging_lag4,avg_time_charging_lag5_median,avg_time_charging_lag6_median,avg_time_charging_lag7_median,avg_time_charging_lag8_median,avg_time_charging_lag9_median,avg_time_charging_lag10_median,avg_time_charging_lag11_median,avg_time_charging_lag12_random,avg_time_charging_lag13_random,avg_time_charging_lag14_random,fail_7
0,0.044546,0.409384,0.080534,0.024585,0.300038,0.543971,0.300916,0.040539,0.040429,0.036791,0.055883,0.024805,0.024729,0.028984,0
1,0.432996,0.041909,0.076458,0.129345,0.061183,0.233001,0.072878,0.320781,0.023063,0.454664,0.374521,0.093281,0.176043,0.030757,0
2,0.049358,0.040683,0.055336,0.267046,0.066037,0.055692,0.060505,0.195630,0.422331,0.473060,0.054999,0.026202,0.022890,0.221858,0
3,0.038870,0.396504,0.441206,0.021194,0.049559,0.292579,0.490225,0.034124,0.022521,0.214760,0.034503,0.021311,0.101397,0.028452,0
4,0.044299,0.431156,0.056695,0.452949,0.186614,0.572724,0.051844,0.234403,0.341880,0.028301,0.050428,0.027949,0.210149,0.176033,0


In [23]:
target = data["fail_7"]
features = data.drop(columns = ["fail_7"])
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

In [28]:
#copied
def training_predict(trainer, X_train, Y_train, X_test, Y_test):
    scores = {} # Initializing a Dictionary to store evaluation metrics
    
    trainer.fit(X_train, Y_train)  # Training
    Y_pred = trainer.predict(X_test) # Predicts
    
    '''
    Stores the result of trainer in its individual Key
    '''
    scores['auc'] = roc_auc_score(Y_test , trainer.predict_proba(X_test)[:,1])
    scores["AP-SCORE"]=average_precision_score(Y_test , trainer.predict_proba(X_test)[:,1])
    
    scores['Recall_test'] = recall_score(Y_test , Y_pred)
    
    scores['Precision_test'] = precision_score(Y_test , Y_pred)
    
    scores['F1_Score_test'] = f1_score(Y_test , Y_pred)
    
    
    return scores

In [29]:
xgb = XGBClassifier(random_state= 42)
rf = RandomForestClassifier(random_state= 42,class_weight='balanced')
gbm=GradientBoostingClassifier(random_state=42)


In [33]:
predictors = {} # Initializing a dictionary to store training and testing data
 
for model in [rf, gbm,xgb]:
    model_name = model.__class__.__name__
    predictors[model_name]  = {}
    pipe=make_pipeline(model)
    predictors[model_name] =  training_predict(pipe, X_train, y_train, X_test, y_test) 
    #Scores that is returned from trainer goes to
    print(predictors)
  


{'RandomForestClassifier': {'auc': 0.9347601140590663, 'AP-SCORE': 0.9490695394987226, 'Recall_test': 0.7733707986598941, 'Precision_test': 0.9623453469607316, 'F1_Score_test': 0.8575708550542274}}
{'RandomForestClassifier': {'auc': 0.9347601140590663, 'AP-SCORE': 0.9490695394987226, 'Recall_test': 0.7733707986598941, 'Precision_test': 0.9623453469607316, 'F1_Score_test': 0.8575708550542274}, 'GradientBoostingClassifier': {'auc': 0.9332864741025441, 'AP-SCORE': 0.9498607546089218, 'Recall_test': 0.8004971360639792, 'Precision_test': 0.9440479225082845, 'F1_Score_test': 0.866366454178607}}
{'RandomForestClassifier': {'auc': 0.9347601140590663, 'AP-SCORE': 0.9490695394987226, 'Recall_test': 0.7733707986598941, 'Precision_test': 0.9623453469607316, 'F1_Score_test': 0.8575708550542274}, 'GradientBoostingClassifier': {'auc': 0.9332864741025441, 'AP-SCORE': 0.9498607546089218, 'Recall_test': 0.8004971360639792, 'Precision_test': 0.9440479225082845, 'F1_Score_test': 0.866366454178607}, 'XGBCl

In [61]:
pd.DataFrame.from_dict(predictors).T

,auc,AP-SCORE,Recall_test,Precision_test,F1_Score_test
RandomForestClassifier,0.934760,0.949070,0.773371,0.962345,0.857571
GradientBoostingClassifier,0.933286,0.949861,0.800497,0.944048,0.866366
XGBClassifier,0.928146,0.946498,0.802767,0.929778,0.861617
